# App for analyzing GECCO runs from the Galaxy

1. Upload local data or query results of the GECCO from the Galaxy.
2. Identifying Biosynthetic Gene Clusters (BGCs).
3. Visualize BGCs.
4. Compare two samples in respect to each other.

Note: Sending GECCO jobs to Galaxy is part of another separate application.

## Platform dependent part
- Resolve platform setup
- the difference to local imports should be resolved by setting the VRE packages well

In [ ]:
import sys
import os
import io

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils import init_setup
init_setup()

In [ ]:
# This needs to be repeated here for the Pannel dashboard to work, WEIRD
# TODO: report as possible bug
import sys
import os
import io

# import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
import panel as pn

# Import
import bioblend.galaxy as g  # BioBlend is a Python library, wrapping the functionality of Galaxy and CloudMan APIs
from bioblend.galaxy import GalaxyInstance

# All low level functions are imported from the momics package
import momics.diversity as div
import momics.plotting as pl
from momics.panel_utils import create_widgets, create_indicators
from momics.helpers import memory_load

# Note: This is breaking the panel preview functionality
# %load_ext autoreload
# %autoreload 2

## Loading and setup

In [3]:
if 'google.colab' in str(get_ipython()):
    root_folder = os.path.abspath(os.path.join('/content/momics-demos'))
else:
    root_folder = os.path.abspath(os.path.join('../'))

assets_folder = os.path.join(root_folder, 'assets')

## App Setup

In [4]:
def display_tables_after_upload(d1, d2=None, d3=None):
    print(d1)
    df1 = pd.DataFrame.from_dict(d1)
    t1 = pn.widgets.Tabulator(df1, sizing_mode="stretch_both", name="Data View")
    return t1

In [ ]:
pn.extension("tabulator", "mathjax", "filedropper")
ACCENT = "teal"

styles = {
    "box-shadow": "rgba(50, 50, 93, 0.25) 0px 6px 12px -2px, rgba(0, 0, 0, 0.3) 0px 3px 7px -3px",
    "border-radius": "4px",
    "padding": "10px",
}

image = pn.pane.JPG(os.path.join(assets_folder, "figs/logo_gecco.jpeg"),
                    width=100, height=100)

markdown_intro = pn.pane.Markdown(
"""
**If you want to use the Galaxy API:**
- you need to provide your credentials.
- and `job_id` of the job that created the files to get those files.
- For comparison study, you need to provide two job_ids.
"""
)

md_upload = pn.pane.Markdown(
"""
**Upload your files:**
You need to select exactly 3 files with endings:
- `_BGC.tsv`
- `_features.tsv`
- `_genes.tsv`
"""
)

md_get_galaxy = pn.pane.Markdown(
"""
**Get files from Galaxy:**
You need to provide your credentials and `job_id`.
"""
)

upload_local = pn.widgets.FileDropper(multiple=True,
                                    #   accepted_filetypes=[".tsv"],
                                      layout="compact")
literal_galaxy_url = pn.widgets.TextInput(
    name='Galaxy server URL',
    placeholder='Enter a https server address here...')


literal_galaxy_key = pn.widgets.PasswordInput(
    name='Password',
    placeholder='Enter your password here...')

table1 = pn.bind(display_tables_after_upload,
                 d1=upload_local,
                 watch=True,
                 )

tabs = pn.Tabs(
    ("Table 1", table1),
    dynamic=True,
    styles=styles, sizing_mode="stretch_width", height=500, margin=10,
)

template = pn.template.FastListTemplate(
    title="DBiosynthetic Gene Cluster Analysis",
    sidebar=[image,
             md_upload, upload_local,
             md_get_galaxy, literal_galaxy_url, literal_galaxy_key,
            #  get_galaxy_files,
            #  pn.layout.Divider(),
            #  "# Beta diversity", select_table_beta, select_taxon, select_beta_factor,
             ],
    main=[pn.Column(markdown_intro,
                    pn.layout.Divider(),
                    tabs,
                    sizing_mode="stretch_both",
                   )],
    main_layout=None,
    accent=ACCENT,
    meta_refresh="2",
)

template.servable()